In [65]:
using Polyhedra, CDDLib
using DrakeVisualizer
import ColorTypes: RGBA
using PyCall
path_to_my_module = "/home/manuelli/drake-mpc/src/boxrobots/src"
push!(LOAD_PATH, path_to_my_module)
reload("BoxRobots")
import BoxRobots
using DrakeVisualizer
import ColorTypes: RGBA
using Polyhedra: SimpleHRepresentation

br = BoxRobots

BoxRobots

In [66]:
# unshift!(PyVector(pyimport("sys")["path"]), "../")
@pyimport boxatlas
@pyimport boxatlas.boxatlascontroller as boxatlascontroller
@pyimport boxatlas.contactstabilizationutils as csu
CSU = csu.ContactStabilizationUtils

PyObject <class boxatlas.contactstabilizationutils.ContactStabilizationUtils at 0x7f38f4501f58>

In [67]:
# attempt to visualize a state
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window();
vis = Visualizer()
delete!(vis)
env, robot = br.make_robot_and_environment()
robot_state = br.make_robot_state()
robot_state.centroidal_dynamics_state.vel[1] = -1.75
br.draw_environment(vis, env)
mass = robot.mass
gravity = robot.gravity
vis_options = br.BoxRobotVisualizerOptions(force_arrow_normalizer=mass*abs(gravity[end]))

BoxRobots.BoxRobotVisualizerOptions
  force_arrow_normalizer: Float64 98.0
  com_radius: Float64 0.1
  contact_point_radius: Float64 0.05
  playback_dt: Float64 0.05


In [68]:
miqp_controller = br.MIQPController()
t = 0.0
dt = 0.05
duration = 1.5
@time traj = br.simulate_tspan(robot, miqp_controller, robot_state, dt, duration)
br.playback_trajectory(vis, traj; options=vis_options)

Optimize a model with 2052 rows, 680 columns and 5318 nonzeros
Model has 1188 quadratic objective terms
Coefficient statistics:
  Matrix range    [2e-03, 1e+03]
  Objective range [2e+00, 2e+03]
  Bounds range    [5e-01, 1e+03]
  RHS range       [2e-01, 1e+03]
Presolve removed 452 rows and 50 columns
Presolve time: 0.01s
Presolved: 1600 rows, 630 columns, 3988 nonzeros
Presolved model has 1144 quadratic objective terms
Variable types: 550 continuous, 80 integer (80 binary)

Root relaxation: objective -1.401342e+03, 1884 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -1401.3422    0   75          - -1401.3422      -     -    0s
H    0     0                    -1352.123576 -1401.3422  3.64%     -    0s
     0     2 -1401.3422    0   75 -1352.1236 -1401.3422  3.64%     -    0s
H  364   245                    -1386.119519 -1394.8235  0.63%   6.7    0

In [69]:
br.playback_trajectory(vis, traj, options=vis_options, playback_speed=1.0)

In [70]:
# extract solve times from the trajectory of SimulationData
function get_solve_time(data)
    return data.controller_data.solve_time
end

function get_contact_array(data)
    return data.state.limb_states[:left_hand].in_contact 
end

solve_time_array = get_solve_time.(traj.data)
contact_array = get_contact_array.(traj.data)

31-element Array{Bool,1}:
 false
 false
  true
  true
  true
  true
  true
  true
  true
  true
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [71]:
using Plots
# Use the GR backend for Plots.jl, because it's fast
gr()

Plots.GRBackend()

In [72]:
plot(traj.time, solve_time_array, xlabel="time", label="solve time")
plot!(traj.time, contact_array, label="contact indicator")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 1.25 
 
 
 1.50 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 time 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,3.93701 49.4523,215.463 68.301,275.424 87.1496,238.935 105.998,238.846 124.847,178.106 143.696,218.508 162.544,273.924 181.393,271.677 200.241,284.416 
 219.09,288.172 237.939,292.852 256.787,288.691 275.636,292.716 294.485,273.242 313.333,276.236 332.182,270.442 351.031,272.967 369.879,267.875 388.728,276.973 
 407.577,283.131 426.425,262.193 445.274,268.967 464.122,259.37 482.971,276.999 501.82,280.306 520.668,271.914 539.517,281.613 558.366,280.832 577.214,250.534 
 596.063,284.06 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,369.674 49.4523,369.674 68.301,117.516 87.1496,117.516 105.998,117.516 124.847,117.516 143.696,117.516 162.544,117.516 181.393,117.516 200.241,117.516 
 219.09,369.674 237.939,369.674 256.787,369.674 275.636,369.674 294.485,369.674 313.333,369.674 332.182,369.674 351.031,369.674 369.879,369.674 388.728,369.674 
 407.577,369.674 426.425,369.674 445.274,369.674 464.122,369.674 482.971,369.674 501.82,369.674 520.668,369.674 539.517,369.674 558.366,369.674 577.214,369.674 
 596.063,369.674 
 "/>
 
 
 
 
 solve time 
 
 
 
 contact indicator